In [ ]:
import datetime
import itertools
from os import path
import time

import ads
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Python was created: 20 February 1991
years = range(1991, 2018+1, 1)
months = range(1, 12+1, 1)
dates = [(y, m) for y, m in itertools.product(years, months) 
         if y < 2018 or (y == 2018 and m < 3)][1:] # Feb 1991

ndates = len(dates) - 1
print('{0} queries to ADS in total'.format(ndates))

In [ ]:
def get_df(term, db, dates, overwrite=False, sleep=0.25):
    cache_file = '{0}_{1}.csv'.format(term, db)
    ndates = len(dates)-1
    q = 'full:"{term:s}"  pubdate:[{y1:d}-{m1:02d}-01 TO {y2:d}-{m2:02d}-01] database:"{db:s}"'
    
    if not path.exists(cache_file) or overwrite:
        df = pd.DataFrame({'date': np.full(ndates, datetime.date(1984, 1, 1)),
                           'count': np.full(ndates, -1)})
        df.to_csv(cache_file, index=False)
        
    df = pd.read_csv(cache_file)
    d = df.to_dict()
    
    search = None
    for i, (y1, m1), (y2, m2) in tqdm(zip(range(ndates), dates[:-1], dates[1:]), 
                                      total=ndates):
        
        if d['count'][i] >= 0:
            continue
        
        search = ads.SearchQuery(q=q.format(y1=y1, y2=y2, m1=m1, m2=m2,
                                            term=term, db=db), 
                                 rows=1000)
        d['count'][i] = len(list(search))
        d['date'][i] = datetime.date(y1, m1, 1)

        time.sleep(sleep)
    
    if search is not None and search.response is not None:
        nremain = search.response.get_ratelimits()["remaining"]
        print('{0} queries remaining'.format(nremain))
    
    df = pd.DataFrame(d)
    df.to_csv(cache_file, index=False)
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [ ]:
py_astro = get_df('python', 'astronomy', dates, sleep=0.1)

In [ ]:
py_phys = get_df('python', 'physics', dates, sleep=0.1)

In [ ]:
idl_astro = get_df('idl', 'astronomy', dates, sleep=0.1)

In [ ]:
matlab_astro = get_df('matlab', 'astronomy', dates, sleep=0.1)

In [ ]:
fortran_astro = get_df('fortran', 'astronomy', dates, sleep=0.1)

In [ ]:
astropy = get_df('astropy', 'astronomy', dates[250:], # only post-2012
                 sleep=0.1) 

In [ ]:
mcmc_astro = get_df('mcmc', 'astronomy', dates, sleep=0.1)

In [ ]:
emcee_astro = get_df('emcee', 'astronomy', dates, sleep=0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(mcmc_astro['date'], mcmc_astro['count'], 
        marker='', drawstyle='steps-mid', color='#333333', lw=1,
        label='"MCMC"')

ax.plot(emcee_astro['date'], emcee_astro['count'], 
        marker='', drawstyle='steps-mid', color='#888888', lw=2,
        label='"emcee"')
    
ax.set_xlim(datetime.date(2011,1,1),
            datetime.date(2018,2,1))

ax.legend(loc='upper left', fontsize=16, framealpha=1)

ax.set_xlabel('time')
ax.set_ylabel('mentions per month')
fig.suptitle('Source: ADS', y=0.02, x=0.99, fontsize=14, 
             fontstyle='italic', ha='right')

ax.set_title('mentions in the astronomical literature')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
dfs = [py_astro, astropy, idl_astro, matlab_astro]
labels = ['"python"', '"astropy"', '"IDL"', '"Matlab"']
styles = [dict(lw=2), dict(color='tab:orange', lw=2), 
          dict(color='tab:red', alpha=0.5), dict(color='tab:green', alpha=0.5)]

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for df, label, style in zip(dfs, labels, styles):
    grouped = df.groupby(df['date'].map(lambda t: t.year)).sum()
    ax.plot(grouped.index, grouped['count'], marker='', #drawstyle='steps-mid',
            label=label, **style)

ax.legend(loc='upper left', fontsize=14, framealpha=1)

ax.set_xlim(1992, 2017)
ax.xaxis.set_ticks(np.arange(1993, 2017+1, 4))

ax.set_xlabel('time')
ax.set_ylabel('mentions per year')
fig.suptitle('Source: ADS', y=0.02, x=0.99, fontsize=14, 
             fontstyle='italic', ha='right')
fig.set_facecolor('w')

ax.set_title('mentions in the astronomical literature')

fig.tight_layout()

In [ ]:
base_style = dict(drawstyle='steps-pre', marker='')

fig, ax = plt.subplots(1, 1, figsize=(12, 6))

ax.plot(py_astro['date'], py_astro['count'], 
        color='k', label='"python" (astronomy)', alpha=1, zorder=100,
        **base_style)
ax.plot(py_phys['date'], py_phys['count'], 
        color='tab:blue', label='"python" (physics)', alpha=0.5,
        **base_style)

ax.plot(astropy['date'], astropy['count'], 
        color='tab:orange', label='"astropy"',
        **base_style)

ax.plot(idl_astro['date'], idl_astro['count'], 
        color='tab:red', label='"IDL"', alpha=0.5,
        **base_style)

ax.legend(loc='upper left', fontsize=18, framealpha=1)
    
# ax.set_yscale('log')

ax.xaxis.set_ticks([datetime.date(x, 1, 1) for x in range(1991, 2018+1, 2)]);
ax.set_xlim(datetime.date(1991,1,1), 
            datetime.date(2018,3,1))

ax.set_xlabel('time')
ax.set_ylabel('mentions per month')
ax.set_title('Source: ADS', fontsize=14)
fig.set_facecolor('w')

Dates of all ADASS and "SPIE Instrumentation" conferences I could find.

In [ ]:
adass = [(1991, 11, 6), (1992, 11, 2), (1993, 10, 13), (1994, 9, 26), 
         (1995, 10, 22), (1996, 9, 22), (1997, 9, 14), (1998, 11, 1), (1999, 10, 3),
         (2000, 11, 12), (2001, 9, 30), (2002, 10, 13), (2003, 10, 12),
         (2004, 10, 24), (2005, 10, 2), (2006, 10, 15), (2007, 9, 23),
         (2008, 11, 2), (2009, 10, 4), (2010, 11, 7), (2011, 11, 6),
         (2012, 11, 4), (2013, 9, 29), (2014, 10, 5), (2015, 10, 25),
         (2016, 10, 16), (2017, 10, 22)]

spie = [(2012, 8, 1), (2014, 6, 1), (2016, 6, 1)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

ax.plot(py_astro['date'], py_astro['count'], 
        drawstyle='steps-pre', marker='', color='#333333')

for i, stuff in enumerate(adass):
    kw = dict()
    if i == 0:
        kw = dict(label='ADASS')
    ax.axvline(datetime.date(*stuff), linestyle='-', marker='', 
               zorder=-100, color='tab:blue', alpha=0.35, **kw)
    
for i, stuff in enumerate(spie):
    kw = dict()
    if i == 0:
        kw = dict(label='SPIE')
    ax.axvline(datetime.date(*stuff), linestyle='-', marker='',
               zorder=-100, color='tab:orange', alpha=0.35, **kw)

ax.legend(loc='upper left', fontsize=18, framealpha=1)
    
ax.set_yscale('log')

ax.xaxis.set_ticks([datetime.date(x, 1, 1) for x in range(1991, 2018+1, 2)]);
ax.set_xlim(datetime.date(1991,1,1), 
            datetime.date(2018,3,1))

ax.set_xlabel('time')
ax.set_ylabel('Python mentions per month')
ax.set_title('Source: ADS', fontsize=14)
fig.set_facecolor('w')